# Demo of fitting individual galaxy SFHs with Diffstar

This notebook illustrates a worked example of how to fit an individual SFH of a simulated galaxy with the diffstar model. For a parallelized script, see `history_fitting_script.py`. First we'll download a very small dataset of a handful of galaxies from the TNG simulation.

In [ ]:
! curl https://portal.nersc.gov/project/hacc/aphearin/diffstar_data/tng_cosmic_time.txt > tng_cosmic_time.txt

In [ ]:
! curl https://portal.nersc.gov/project/hacc/aphearin/diffstar_data/tng_diffstar_example_data.h5 > tng_diffstar_example_data.h5

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
import h5py

fn_example_data = 'tng_diffstar_example_data.h5'
tng_data = dict()
with h5py.File(fn_example_data, 'r') as hdf:
    for key in hdf.keys():
        tng_data[key] = hdf[key][...]
tng_data.keys()
n_halos = len(tng_data['halo_id'])

In [ ]:
from diffstar.utils import _jax_get_dt_array

tarr = np.loadtxt('tng_cosmic_time.txt')
dtarr = _jax_get_dt_array(tarr)

## Pick a particular example galaxy history to fit

In [ ]:
from diffstar.fitting_helpers.fit_smah_helpers import get_loss_data_default
itest = 0

halo_id = tng_data['halo_id'][itest]
lgsmah = tng_data['logsmh_sim'][itest]
sfrh = tng_data['sfh_sim'][itest]
mah_params = tng_data['mah_fit_params'][itest]
logmp_halo = tng_data['logmp'][itest]

p_init, loss_data = get_loss_data_default(
    tarr, dtarr, sfrh, lgsmah, logmp_halo, mah_params
)

## Use L-BFGS-B to fit the SFH with diffstar

In [ ]:
from diffstar.fitting_helpers.fit_smah_helpers import loss_default, loss_grad_default_np
from diffstar.fitting_helpers.utils import minimizer_wrapper


_res = minimizer_wrapper(
    loss_default, loss_grad_default_np, p_init, loss_data, 
)
p_best, loss_best, success = _res

## Grab the unbounded values of the best-fit parameters

In [ ]:
from diffstar.fitting_helpers.fit_smah_helpers import get_outline_default, get_header

outline = get_outline_default(halo_id, loss_data, p_best, loss_best, success)
header = get_header()

output_data = dict(zip(header[1:].strip().split(), outline.strip().split()))

colnames = list(output_data.keys())
sfr_colnames = colnames[1:6]
q_colnames = colnames[6:10]

u_sfr_fit_params = np.array([output_data[key] for key in sfr_colnames]).astype(float)
u_q_fit_params = np.array([output_data[key] for key in q_colnames]).astype(float)

output_data

## Transform the unbounded parameters to the actual `diffstar` parameters

In [ ]:
from diffstar.kernels.main_sequence_kernels import _get_bounded_sfr_params
from diffstar.kernels.quenching_kernels import _get_bounded_q_params

sfr_fit_params = np.array(_get_bounded_sfr_params(*u_sfr_fit_params))
q_fit_params = np.array(_get_bounded_q_params(*u_q_fit_params))

## Calculate histories using the best-fit model

In [ ]:
from diffstar.fitting_helpers.fitting_kernels import calculate_sm_sfr_fstar_history_from_mah

dmhdt_fit, log_mah_fit = loss_data[2:4]
lgt = np.log10(tarr)
index_select, index_high, fstar_tdelay = loss_data[8:11]
    
_histories = calculate_sm_sfr_fstar_history_from_mah(
    lgt,
    dtarr,
    dmhdt_fit,
    log_mah_fit,
    u_sfr_fit_params,
    u_q_fit_params,
    index_select,
    index_high,
    fstar_tdelay,
)
smh_fit, sfh_fit, fstar_fit = _histories

## Compare the model to the simulated SFH

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,4), sharex=True)


ax[0].plot(tarr, 10**lgsmah, label='IllustrisTNG')
ax[0].plot(tarr, smh_fit, ls='--', label='diffstar')
ax[0].set_yscale('log')
ax[0].set_ylim(1e8, 1e12)
ax[0].set_xlim(0, 14)
ax[0].set_xlabel('Cosmic time [Gyr]')
ax[0].set_ylabel('$M_\star [M_{\odot}]$')
ax[0].legend(loc=4)

ax[1].plot(tarr, sfrh)
ax[1].plot(tarr, sfh_fit, ls='--')
ax[1].set_yscale('log')
ax[1].set_ylim(1e-1, 5e2)
ax[1].set_xlim(0, 14)
ax[1].set_xticks(np.arange(1,14,2))
ax[1].set_xlabel('Cosmic time [Gyr]')
ax[1].set_ylabel('$dM_\star/dt \,[M_{\odot}/yr]$')
fig.subplots_adjust(wspace=0.3)

plt.show()

## Show the model main sequence efficiency and quenching function

In [ ]:
from diffstar.kernels.main_sequence_kernels import _sfr_eff_plaw
from diffstar.kernels.quenching_kernels import _quenching_kern_u_params

_m = np.linspace(10, 14, 100)
MS_efficiency_fit = _sfr_eff_plaw(_m, *sfr_fit_params[:-1])
_t = np.linspace(-1,2,1000)
qf = _quenching_kern_u_params(_t, *u_q_fit_params)

fig, ax = plt.subplots(1,2, figsize=(10,4), sharex=False)


ax[0].plot(_m, MS_efficiency_fit)
ax[0].set_yscale('log')
ax[0].set_ylim(1e-2, 1.5)
ax[0].set_xlim(10, 13)
ax[0].set_xticks(np.arange(10,13.1,1.0))
ax[0].set_xlabel('$\log M_{h}(t) \, [M_{\odot}]$')
ax[0].set_ylabel('$\epsilon(M_{h}(t))$')

ax[1].plot(10**_t, qf)
ax[1].set_xlim(0, 14)
ax[1].set_ylim(0, 1.05)
ax[1].set_xticks(np.arange(1,14,2))
ax[1].set_xlabel('Cosmic time [Gyr]')
ax[1].set_ylabel('$F_q(t)$')
fig.subplots_adjust(wspace=0.3)
